In [1]:
import muspy
import numpy as np
import math

In [34]:
test = muspy.load("../data/example.json")

In [126]:
def get_resolution_threshold(mus, threshold=100):
    """Return the nearest even MusPy resolution needed to consider notes 'threshold' milliseconds apart as different (but no lower)."""
    return [round(((x.qpm / 60) ** -1 * 1000 / threshold) / 2) * 2 for x in mus.tempos]

def parse_input(true_data, path, threshold=100, resolution=None):
    """Parse a .txt file output from the follow.js API into a MusPy object."""
    'TODO: Add support for multiple tempos'
    with open(path) as f:
        raw_data = f.readlines()[1:]

    raw_data = np.array([[float(y) for y in x.strip("\n|,").split(",")] for x in raw_data]) # raw_data is now a list of (time, pitch, velocity) tuples
    parsed_data = np.zeros((raw_data.shape[0] // 2, 4))
    countoff_offset = true_data.time_signatures[0].numerator * (true_data.tempos[0].qpm / 60) ** -1 # time offset in milliseconds to account for countoff

    raw_res = get_resolution_threshold(true_data, threshold)[0]
    if resolution is not None:
        raw_res = resolution
    time2beats = lambda x: round(x * true_data.tempos[0].qpm * raw_res / 60)
    for i in np.arange(0, raw_data.shape[0], 2):
        parsed_data[i // 2] = np.array(
            [
                time2beats(raw_data[i][0] / 1000 - countoff_offset), # time in new resolution
                raw_data[i][1], # MIDI pitch
                time2beats((raw_data[i+1][0] - raw_data[i][0]) / 1000), # duration in new resolution
                raw_data[i][2] # velocity
            ],
        dtype=int)
    input_mus = muspy.from_note_representation(parsed_data.astype(int), resolution=raw_res)
    return input_mus

In [127]:
def clean_output(true_path, out_path, threshold=100, resolution=None):
    """Clean a .txt file output from the follow.js API into a MusPy object."""
    if true_path.endswith(".json"):
        true_mus = muspy.load(true_path)
    elif true_path.endswith(".xml"):
        true_mus = muspy.read_musicxml(true_path)
    elif true_path.endswith(".abc"):
        true_mus = muspy.read_abc(true_path)
    else:
        raise NotImplementedError("File type not supported.")
    
    if type(true_mus) == list:
        true_mus = true_mus[0]
    input_mus = parse_input(true_mus, out_path, threshold, resolution)
    finest_rhythm = round(sorted(np.unique((true_mus.to_note_representation()[:, 0] / true_mus.resolution) % 1))[1] ** -1) # the finest rhythm in the piece
    true_mus.adjust_resolution(target=math.lcm(input_mus.resolution, finest_rhythm))
    input_mus.adjust_resolution(target=math.lcm(input_mus.resolution, finest_rhythm))
    input_mus.barlines = true_mus.barlines
    input_mus.time_signatures = true_mus.time_signatures
    bartimes = np.array([x.time for x in input_mus.barlines])
    bartime2n = {x: i for i,x in enumerate(bartimes)}
    for i, note in enumerate(input_mus.tracks[0].notes):
        note.label = int(
            note.time in true_mus.to_note_representation()[:,0] and note.pitch == true_mus.to_note_representation()[np.where(note.time == true_mus.to_note_representation()[:, 0]),1] # Label is 1 if the note is correct at resolution
        )
   
        note.bar_n = bartime2n[bartimes[bartimes <= note.time].max()] # Label with 0-indexed bar number
        if note.label == 1:
            print(note, note.label, note.bar_n)
    return input_mus

In [128]:
x =clean_output("/Users/znovack/opt/miniconda3/envs/music/lib/python3.9/site-packages/music21/corpus/ciconia/quod_jactatur.xml", '/Users/znovack/Downloads/inputs/quod_jactatur_output (2).txt', resolution=2)

Note(time=0, pitch=60, duration=0, velocity=113) 1 0
Note(time=3, pitch=62, duration=0, velocity=89) 1 0
Note(time=4, pitch=64, duration=0, velocity=103) 1 1
Note(time=5, pitch=62, duration=0, velocity=84) 1 1
Note(time=6, pitch=60, duration=0, velocity=87) 1 1
Note(time=8, pitch=60, duration=0, velocity=91) 1 2
Note(time=10, pitch=58, duration=1, velocity=73) 1 2
Note(time=12, pitch=57, duration=2, velocity=82) 1 3
Note(time=16, pitch=62, duration=1, velocity=80) 1 4
Note(time=18, pitch=58, duration=1, velocity=50) 1 4
Note(time=19, pitch=57, duration=0, velocity=66) 1 4
Note(time=20, pitch=55, duration=3, velocity=69) 1 5
Note(time=24, pitch=62, duration=1, velocity=81) 1 6
Note(time=26, pitch=64, duration=1, velocity=87) 1 6
Note(time=30, pitch=60, duration=0, velocity=64) 1 7
Note(time=30, pitch=60, duration=1, velocity=91) 1 7
Note(time=42, pitch=64, duration=0, velocity=76) 1 10
Note(time=48, pitch=64, duration=0, velocity=88) 1 12
Note(time=51, pitch=60, duration=0, velocity=87)

In [125]:
x.time_signatures

[TimeSignature(time=0, numerator=2, denominator=4)]